In [35]:
from Bio import PDB
import numpy as np
import io
import pandas as pd

In [26]:
seq = "MSEEAQNDPLLPGYSFNAHLVAGLTPIEANGYLDFFIDRPLGMKGYILNLTIRGQGVVKNQGREFVCRPGDILLFPPGEIHHYGRHPEAREWYHQWVYFRPRAYWHEWLNWPSIFANTGFFRPDEAHQPHFSDLFGQIINAGQGEGRYSELLAINLIEQLLLRRMEAINESSGLATTLERIEKNFVITDPRLPDNPIIFASDSFLQLTEYSREEILGRNCRFLQGPETDRATVRKIRDAIDNQTEVTVQLINYTKSGKKFWNLFHLQPMRDQKGDVQYFIGVQLDGTEHVRDAAEREGVMLIKKTAENIDEAAKGSLHPPMDNRVREACQYISDHLADSNFDIASVAQHVCLSPSRLSHLFRQQLGISVLSWREDQRISQAKLLLSTTRMPIATVGHNVGFDDQLYFSRVFKKCTGASPSEFRAGCEEKVNDVAVKLSG"

In [27]:
array = np.random.randint(0, 100, len(seq))

In [65]:
array = pd.read_csv("output/Mut1_coverage.csv", header=None).values.flatten()
array = np.round(array, 5) * 100

In [28]:
def overwrite_b_factors(pdb_str: str, bfactors: np.ndarray) -> str:
    """From the original AF2 github: Overwrites the B-factors in pdb_str with contents of bfactors array."""

    parser = PDB.MMCIFParser(QUIET=True)
    handle = io.StringIO(pdb_str)
    structure = parser.get_structure('', handle)

    curr_resid = ('', '', '')
    idx = -1
    for atom in structure.get_atoms():
        atom_resid = atom.parent.get_id()
        if atom_resid != curr_resid:
            idx += 1
            if idx >= bfactors.shape[0]:
                raise ValueError('Index into bfactors exceeds number of residues. '
                                 'B-factors shape: {shape}, idx: {idx}.')
        curr_resid = atom_resid
        atom.bfactor = bfactors[idx]

    new_pdb = io.StringIO()
    pdb_io = PDB.PDBIO()
    pdb_io.set_structure(structure)
    pdb_io.save(new_pdb)
    return new_pdb.getvalue()

In [66]:
path = '/Users/annavonbachmann/Downloads/fold_2025_02_05_13_34_my_first_job/fold_2025_02_05_13_34_my_first_job_model_0.cif'

In [67]:
with open(path,'r') as f:
    pdb_str = f.read()
    

In [23]:
# parser = PDB.MMCIFParser(QUIET=True)
# handle = io.StringIO(pdb_str)
# structure = parser.get_structure('', handle)

In [68]:
new_pdb_str = overwrite_b_factors(pdb_str, array)

In [69]:
with open('/Users/annavonbachmann/Downloads/fold_2025_02_05_13_34_my_first_job/fold_2025_02_05_13_34_my_first_job_model_0_Mut1_coverage.pdb', 'w') as f:
    f.write(new_pdb_str)